In [5]:
import pandas as pd
from math import pi

# Rate of kilonovae, from LIGO (ensure rate is in the correct units per year - it is!)
##  105.5 (+190.2 or −83.9) Gpc^−3*yr^−1
# in https://arxiv.org/pdf/2111.03634

#Rate
upper_rate = 295.7
median_rate = 105.5
lower_rate = 21.6  # Assuming this is already per year

# A distance away from center (radius), for a detection. Set range to 10-600 Mpc. Put into Gpc. 

#R_min = 10
#R_max = 600
#R_min = 0.0099999987036885
#R_max = 0.59999992222131
#R = 1200
R = 1.19999984444262 # Gpc

# Check if LSST's footprint is the same, contact Lynne Jones if necessary
# on survey_footprint.ipynb notebook on git
Area_Footprint = 20283  # Ensure this is in square degrees

# Find the surface area of a sphere (assuming units are correct, Area of sphere is 41252.96 sq deg)
A_sphere = 41252.96


# Assuming V_sphere is the volume of a sphere with the radius R
# Define or calculate V_sphere if needed, here's an example if R is radius
V_sphere = (4/3) * pi * (R ** 3)

# Find volume of a sector of a sphere
#Per conversation with Igor: area_footprint = area_sector, area_sector = 1/2 * area_sphere
#V_sector is the comoving volume. Basically, R will be a comoving distance, and not a luminosity distance.

# V_sector = V_sphere * Area_Footprint/A_sphere
V_sector = V_sphere * (Area_Footprint / A_sphere)

# Time, over 10 years, length of the survey
T = 10

# Efficiency 
csv_path = "../Stuff/MIL_STRAT/MIL_efficiency_baseline_v3.4_10yrs_cadence.csv"
try:
    e_csv = pd.read_csv(csv_path)
    e = e_csv.iloc[0, 5]  
    # Assuming the efficiency value is in the 6th row and the first column, prints to check correct efficinecy file is called
    print(e)
except FileNotFoundError:
    print(f"File not found: {csv_path}")
    e = 1  # Default efficiency to 1 (or another appropriate value)
except IndexError:
    print("Index error when accessing the efficiency value.")
    e = 1  # Default efficiency to 1 (or another appropriate value)

# Calculate the expected number of kilonovae
if not e_csv.empty:
    for index, row in e_csv.iterrows():
        efficiency = row[5]
        metric = row[2]
        N = median_rate * V_sector * efficiency * T
        print(f"Expected number of kilonovae for {metric}: {N}")
else:
    print("No efficiency data available for calculations.")



0.000785
Expected number of kilonovae for  KNePopMetric__blue_color_detect: 2.947347680328437
Expected number of kilonovae for  KNePopMetric__multi_color_detect: 37.60965823420376
Expected number of kilonovae for  KNePopMetric__multi_detect: 45.43045469041285
Expected number of kilonovae for  KNePopMetric__red_color_detect: 2.8497285214895336
Expected number of kilonovae for  KNePopMetric__ztfrest_simple: 4.873448775880651
Expected number of kilonovae for  KNePopMetric__ztfrest_simple_blue: 3.3077876514259277
Expected number of kilonovae for  KNePopMetric__ztfrest_simple_red: 1.92985567858448


# Looping

In [14]:
import pandas as pd
from math import pi
import os
import json
from collections import defaultdict

# Rate of kilonovae, from LIGO (ensure rate is in the correct units per year)
#upper_rate = 295.7
#median_rate = 105.5
#lower_rate = 21.6 
rate = 21.6

# Radius
R_Mpc = 1200
R = 1.19999984444262 # Gpc

# Find the surface area of a sphere (assuming units are correct)
A_sphere = 41252.96

# Check if LSST's footprint is the same, contact Lynne Jones if necessary
Area_Footprint = 20283  # Ensure this is in square degrees

# Assuming V_sphere is the volume of a sphere with the radius R
V_sphere = (4/3) * pi * (R ** 3)

# Find volume of a sector of a sphere
#V_sector is the comoving volume. Basically, R will be a comoving distance, and not a luminosity distance.

V_sector = V_sphere * (Area_Footprint / A_sphere)

# Time, over 10 years, length of the survey
T = 10

# Directory containing the efficiency CSV files
directory_path = "../Stuff/MIL_STRAT"

# Dictionary to hold results by cadence family
results = defaultdict(lambda: {
    "run_names": [],
    "metrics": defaultdict(list)
})

# Process each CSV file in the directory
for filename in os.listdir(directory_path):
    if filename.endswith(".csv"):
        # Construct full file path
        csv_path = os.path.join(directory_path, filename)
        
        # Read the CSV file
        try:
            e_csv = pd.read_csv(csv_path)
        except FileNotFoundError:
            print(f"File not found: {csv_path}")
            continue  # Skip to the next file
        except pd.errors.EmptyDataError:
            print(f"No data found in the CSV file: {csv_path}")
            continue  # Skip to the next file

        # Extract the cadence family name
        base_name = filename.replace('MIL_efficiency_', '').replace('_cadence.csv', '')
        cadence_family = base_name.split('_')[0]

        # Print cadence family if not already in results
        if cadence_family not in results:
            print(f"\nCadence Family: {cadence_family}")

        # Append the base_name (filename without prefix and suffix) to run_names
        results[cadence_family]["run_names"].append(base_name)

        # Check if the file has data
        if not e_csv.empty:
            # Print the efficiency for the first row in the file
            e = e_csv.iloc[0, 5]
            print(f"  {base_name}, Efficiency: {e}")

            # Calculate the expected number of kilonovae for each metric
            for index, row in e_csv.iterrows():
                efficiency = row[5]  # Efficiency is in the 6th column
                metric = row[2]  # Metric is in the 3rd column
                N = rate * V_sector * efficiency * T
                # Format metric name to match the JSON format
                metric_key = f"{metric}"
                results[cadence_family]["metrics"][metric_key].append(N)
                print(f"    {metric}: {N}")
        else:
            print(f"  No efficiency data available in file: {base_name}")

# Convert metrics from lists of values to summary_means
for cadence_family in results:
    for metric_key in results[cadence_family]["metrics"]:
        results[cadence_family]["metrics"][metric_key] = {
            "summary_means": results[cadence_family]["metrics"][metric_key]
        }

# Construct the filename dynamically
output_filename = f"realistic_KNe_{R_Mpc}_{int(rate)}.json"
output_file = os.path.join("../Stuff", output_filename)

# Save the results to a JSON file
with open(output_file, 'w') as f:
    json.dump(results, f, indent=4)

print(f"\nResults saved to {output_file}")



Cadence Family: baseline
  baseline_v3.0_10yrs, Efficiency: 0.00075
     KNePopMetric__blue_color_detect: 0.5765331291251329
     KNePopMetric__multi_color_detect: 6.843832598134743
     KNePopMetric__multi_detect: 8.517316094275296
     KNePopMetric__red_color_detect: 0.48582525014277855
     KNePopMetric__ztfrest_simple: 0.852500320266363
     KNePopMetric__ztfrest_simple_blue: 0.5880637917076355
     KNePopMetric__ztfrest_simple_red: 0.3382327690867446
  baseline_v3.4_10yrs, Efficiency: 0.000785
     KNePopMetric__blue_color_detect: 0.6034380084843057
     KNePopMetric__multi_color_detect: 7.700176472595275
     KNePopMetric__multi_detect: 9.301401149885477
     KNePopMetric__red_color_detect: 0.5834515266746345
     KNePopMetric__ztfrest_simple: 0.9977866688058966
     KNePopMetric__ztfrest_simple_blue: 0.6772342490123228
     KNePopMetric__ztfrest_simple_red: 0.3951173711604244
  baseline_v3.3_10yrs, Efficiency: 0.000841
     KNePopMetric__blue_color_detect: 0.6464858154589823
  